In [1]:
import pandas as pd
import numpy as np

In [2]:
# Define number of events in data
number_of_events_ = 100

## Load the DataFrames Stored during Data Preprocessing

In [3]:
'''Refer to Data Preprocessing Notebook to Understand the Semantics and Column Headers'''

'Refer to Data Preprocessing Notebook to Understand the Semantics and Column Headers'

In [4]:
# TODO: Set new string names of column to avoid using b'' syntax to access the rows and columns
def decode_byte_headers(df_=None): 
    if df_ is not None:
        new_names = []
        for column_name in df_.columns:
            new_names.append(str(column_name.decode()))
        df_.columns = new_names
        return df_
    pass

In [5]:
%%time
track_global_df_ = decode_byte_headers(pd.read_msgpack('data/track_global_df_.msgpack'))
track_param_global_df_ = decode_byte_headers(pd.read_msgpack('data/track_param_global_df_.msgpack'))
rechit_global_df_ = decode_byte_headers(pd.read_msgpack('data/rechit_global_df_.msgpack'))
rechit_param_global_df_ = decode_byte_headers(pd.read_msgpack('data/rechit_param_global_df_.msgpack'))

CPU times: user 571 ms, sys: 46.9 ms, total: 618 ms
Wall time: 696 ms


In [6]:
intermediate_df_ = track_param_global_df_[track_param_global_df_['track_eta'] <= 0.9]
intermediate_df_ = intermediate_df_[intermediate_df_['track_pt'] <= 10]
intermediate_df_ = intermediate_df_[intermediate_df_['track_pt'] >= 1]
track_param_global_df_ = intermediate_df_[intermediate_df_['track_eta'] >= -0.9]
track_global_df_ = track_global_df_.iloc[track_param_global_df_['track_id']]

In [7]:
# %%time
# This cell has been rendered obsolete and is only run to check for backward compatibility
# For the other notebook titled 'Data Preprocessing ...'
'''
track_count_ = pd.DataFrame.to_dict(pd.read_csv('data/track_count_.csv'))
rechit_count_ = pd.DataFrame.to_dict(pd.read_csv('data/rechit_count_.csv'))
track_ids_ = pd.DataFrame.to_dict(pd.read_csv('data/track_ids_.csv'))
rechit_ids_ = pd.DataFrame.to_dict(pd.read_csv('data/rechit_ids_.csv'))
'''

"\ntrack_count_ = pd.DataFrame.to_dict(pd.read_csv('data/track_count_.csv'))\nrechit_count_ = pd.DataFrame.to_dict(pd.read_csv('data/rechit_count_.csv'))\ntrack_ids_ = pd.DataFrame.to_dict(pd.read_csv('data/track_ids_.csv'))\nrechit_ids_ = pd.DataFrame.to_dict(pd.read_csv('data/rechit_ids_.csv'))\n"

## Create the Graph Tuples from Hit Matches

In [8]:
from sklearn.preprocessing import StandardScaler

'''
Graphs are created on a per-event basis. We build an adjacency matrix of rechits for each individual event.
The rechit connections are defined by the track that they belong to.
The target label for each node is taken as the first tp_index in its rechit_tp_list

#TODO: Incorporate a more flexible labeling schema - can you label the edges using the 'extra' tp index?
Can this learn more interesting structures for the graph(s)?
# Solution: Use the same node in different graphs - same as above but implementation-wise easier to do.

#TODO: What information do we use to weight the edges in the graph?
Differences in rechit parameters?
Rechit vs. Track Parameters?
'''

"\nGraphs are created on a per-event basis. We build an adjacency matrix of rechits for each individual event.\nThe rechit connections are defined by the track that they belong to.\nThe target label for each node is taken as the first tp_index in its rechit_tp_list\n\n#TODO: Incorporate a more flexible labeling schema - can you label the edges using the 'extra' tp index?\nCan this learn more interesting structures for the graph(s)?\n# Solution: Use the same node in different graphs - same as above but implementation-wise easier to do.\n\n#TODO: What information do we use to weight the edges in the graph?\nDifferences in rechit parameters?\nRechit vs. Track Parameters?\n"

In [9]:
%%time
import numpy as np
from sklearn.preprocessing import StandardScaler

'''TODO: Convert keys to string instead of bytes and check overheads'''

data_dict_list_ = []
scaler = StandardScaler()


# Global Features are track-based so they vary in length per-event
# We find the maximum number of tracks that correspond to max_len of global feature vector
# Is it a good idea to zero-pad global feature vectors less than max_len?
GLOBAL_FEATURES_LEN_ = max([
    len(
        track_global_df_[track_global_df_['event_id']==event_id_]
    ) for event_id_ in range(100)
])

for event_id_ in range(number_of_events_):
    data_dict_ = {}
    senders_ = []
    receivers_ = []
    
    track_event_df_ = track_global_df_[track_global_df_['event_id'] == event_id_]
    track_param_df_ = track_param_global_df_.loc[track_event_df_['track_id']]
    track_df_ = track_event_df_.merge(track_param_df_)

    # Sort the tracks according to increasing track_eta and associate a label with each track
    track_df_.sort_values('track_eta', ascending=True, inplace=True)
    track_df_.index = pd.RangeIndex(len(track_df_.index))  

    rechit_event_df_ = rechit_global_df_[rechit_global_df_['event_id']==event_id_]
    rechit_param_df_ = rechit_param_global_df_[rechit_param_global_df_['event_id']==event_id_]
    rechit_param_df_.index = pd.RangeIndex(len(rechit_param_df_.index))  
    if len(rechit_event_df_) != len(rechit_param_df_):
        print("Error - param data and event data are not of equal length!")
    
    number_of_rechits_in_event_ = len(rechit_event_df_)
    
    # Set the node features as the track features that they belong to
    node_labels_ = np.array(rechit_param_df_['rechit_local_id'].tolist()).astype(int)
    
    # Originally, we were setting node-level features based on the nodes but that can be done
    # for the test data set; instead here we can set the node-level features as the track features
    # at least for training and "learn" the track-level features (eta) based on which we can cluster the nodes?
    # The question still remains how do we initialize the edges ???
    
    # Update: Reverting to node-level features for each node as of now 
    # Modify it to combine some form of track-level features (target?)
    if len(node_labels_) == 0:
        continue
    rechit_feature_vector_ = np.transpose(np.array([rechit_param_df_['rechit_r'].tolist(),
                                  rechit_param_df_['rechit_eta'].tolist(),
                                  rechit_param_df_['rechit_phi'].tolist()]))
    rechit_feature_vector_ = scaler.fit_transform(rechit_feature_vector_)
    node_feature_vector_ = rechit_feature_vector_
    
    # Edge Features uses the track matches to define edge features for each set of nodes
    # We are not relying on edges for now but this will be relevant for message-passing in graph neural networks
    edge_feature_vector_ = []
    
    # For each track, append to the list of source nodes, destination nodes, and node feature vectors
    # Keep track of the node label equal to len(track_df_)
    node_label_i_ = 0
    
    for row in track_df_.itertuples():
        track_edge_features_ = []
        # The itertuples() method for dataframes requires acccess by rows
        # The matched_rechit_ids array is present in the third column of this 'row'
        track_rechit_id_array_ = row[2]
        #print (track_rechit_id_array_)
        src_vertices_ = []
        dest_vertices_ = []
        # Sort the rechits based on values of Rechit R
        # Start track building from the inside and move all the way outside
        final_rechits_ = sorted(track_rechit_id_array_, 
                                key=lambda hit_: rechit_param_df_.loc[int(hit_)]['rechit_r'])
        final_rechits_ = [int(hit_) for hit_ in final_rechits_]
        if len(final_rechits_) < 2:
            # Increment the node label
            node_label_i_ += 1
            continue
        elif len(final_rechits_) == 2:
            src_vertices_.append(final_rechits_[0])
            dest_vertices_.append(final_rechits_[1])
        
        # In order to extend this to 2 skip-connections (expanding to the assumption that 3 hits can be 
        # on the same layer, thus all of them should be connected to a hit on the next layer)
        # Create another else case for len(final_rechits_) == 3: and add the corresponding vertices
        # to src and dest arrays. Then you can modify the addition procedure to include src+[3:] and dest+[:-3]
        # So you will have (1,2), (1,3), and (1,4) edges as a simple example of adding 2-skip-connections
        else:
            # Add the edges starting from node a and going to both a+1 and a+2
            # We define this as 1-skip-connection because hits might lie on the same layer
            # We originally sort them by the radius to ensure skip-connections have a meaning
            src_vertices_.extend(final_rechits_[:-1]+final_rechits_[:-2])
            dest_vertices_.extend(final_rechits_[1:]+final_rechits_[2:])
        # Increment the node label
        node_label_i_ += 1
        senders_.extend(src_vertices_)
        receivers_.extend(dest_vertices_)
        # Define the edge feature vectors of same length as number of vertices
        # Indices 7, 8, 9 correspond to track eta, phi, and qoverp values respectively
        track_edge_features_ = [[row[7], row[8], row[9]]] * len(src_vertices_)
        edge_feature_vector_.append(track_edge_features_)
    
    if node_label_i_ != len(track_df_):
        print("Error: Node Labels don't match the number of tracks - spurious labels generated?", node_label_i_, len(track_df_))
    
    # Define a zero-padded global feature vector
    if len(track_df_) < GLOBAL_FEATURES_LEN_:
        global_feature_vector_ = track_df_['track_eta'].tolist() + [0]*(GLOBAL_FEATURES_LEN_ - len(track_df_))
    else: 
        global_feature_vector_ = track_df_['track_eta'].values
    
    if len(track_edge_features_) != len(dest_vertices_):
        print(len(track_edge_features_), len(dest_vertices_))
        print("Edge features and number of destination edges do not match in event", event_id_)
    
    data_dict_ = {
    "nodes": node_feature_vector_,
    "edges": edge_feature_vector_,
    "senders": senders_,
    "receivers": receivers_
    }
    data_dict_list_.append(data_dict_)
print(len(data_dict_list_), "graphs generated from data")

100 graphs generated from data
CPU times: user 6.43 s, sys: 59.6 ms, total: 6.49 s
Wall time: 6.61 s


## Tensorflow Models

In [10]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from graph_nets import blocks
from graph_nets import graphs
from graph_nets import modules
from graph_nets import utils_np
from graph_nets import utils_tf

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import sonnet as snt
import tensorflow as tf

In [11]:
def print_graphs_tuple(graphs_tuple):
  print("Shapes of `GraphsTuple`'s fields:")
  print(graphs_tuple.map(lambda x: x if x is None else x.shape, fields=graphs.ALL_FIELDS))
  print("\nData contained in `GraphsTuple`'s fields:")
  print("globals:\n{}".format(graphs_tuple.globals))
  print("nodes:\n{}".format(graphs_tuple.nodes))
  print("edges:\n{}".format(graphs_tuple.edges))
  print("senders:\n{}".format(graphs_tuple.senders))
  print("receivers:\n{}".format(graphs_tuple.receivers))
  print("n_node:\n{}".format(graphs_tuple.n_node))
  print("n_edge:\n{}".format(graphs_tuple.n_edge))


In [14]:
# Convert the list of dicts into individua graphs
# This *should* require a placeholder in order to manage easier padding (in the future?)
# There is an error with the graph creation process so placeholders might be the solution to it

tracking_graphs_tuple = utils_np.data_dicts_to_graphs_tuple(data_dict_list_)
print_graphs_tuple(tracking_graphs_tuple)

Shapes of `GraphsTuple`'s fields:
GraphsTuple(nodes=(152630, 3), edges=(1403,), receivers=(36507,), senders=(36507,), globals=None, n_node=(100,), n_edge=(100,))

Data contained in `GraphsTuple`'s fields:
globals:
None
nodes:
[[-1.47861141 -0.41633375 -0.18927357]
 [-1.47073991 -0.40603532 -0.0823086 ]
 [-1.49154642 -0.43907495 -0.01782528]
 ...
 [ 1.68860182  0.89060974 -0.55854773]
 [ 1.81377641  0.86950233 -0.28889089]
 [ 1.69075382  0.30055632 -0.17546064]]
edges:
[list([[0.01188118050492032, 1.622445581574634, 0.15238807366701482], [0.01188118050492032, 1.622445581574634, 0.15238807366701482], [0.01188118050492032, 1.622445581574634, 0.15238807366701482], [0.01188118050492032, 1.622445581574634, 0.15238807366701482], [0.01188118050492032, 1.622445581574634, 0.15238807366701482], [0.01188118050492032, 1.622445581574634, 0.15238807366701482], [0.01188118050492032, 1.622445581574634, 0.15238807366701482], [0.01188118050492032, 1.622445581574634, 0.15238807366701482], [0.0118811805049

In [13]:
''' Manually attempting to create a TF Dataset for TFRecords in DeepHGCal
dataset = tf.data.Dataset.from_generator(lambda: data_dict_list_, tf.int32, output_shapes=[None])
iterator = dataset.make_one_shot_iterator()

next_element = iterator.get_next()

with tf.Session() as sess:
  print(sess.run(next_element))
  print(sess.run(next_element))
'''

InvalidArgumentError: TypeError: `generator` yielded an element that could not be converted to the expected type. The expected type was int32, but the yielded element was {'nodes': array([[-1.47861141, -0.41633375, -0.18927357],
       [-1.47073991, -0.40603532, -0.0823086 ],
       [-1.49154642, -0.43907495, -0.01782528],
       ...,
       [ 2.03664698,  0.44240481, -1.10995009],
       [ 2.03702663,  1.67092634, -0.30561044],
       [ 2.17684212,  1.62728869, -0.28945462]]), 'edges': [[[0.01188118050492032, 1.622445581574634, 0.15238807366701482], [0.01188118050492032, 1.622445581574634, 0.15238807366701482], [0.01188118050492032, 1.622445581574634, 0.15238807366701482], [0.01188118050492032, 1.622445581574634, 0.15238807366701482], [0.01188118050492032, 1.622445581574634, 0.15238807366701482], [0.01188118050492032, 1.622445581574634, 0.15238807366701482], [0.01188118050492032, 1.622445581574634, 0.15238807366701482], [0.01188118050492032, 1.622445581574634, 0.15238807366701482], [0

In [ ]:
graphs_nx = utils_np.graphs_tuple_to_networkxs(tracking_graphs_tuple)
_, axs = plt.subplots(ncols=7, figsize=(8, 8))
for iax, (graph_nx, ax) in enumerate(zip(graphs_nx, axs)):
  nx.draw(graph_nx, ax=ax)
  ax.set_title("Graph {}".format(iax))

In [ ]:
print(i for i in tracking_graphs_tuple)

In [ ]:
tf.reset_default_graph()

seed = 2
rand = np.random.RandomState(seed=seed)

# Model parameters.
# Number of processing (message-passing) steps.
num_processing_steps_tr = 10
num_processing_steps_ge = 10

# Training loss.
loss_ops_tr = create_loss_ops(target_ph, output_ops_tr)

# Loss across processing steps.
loss_op_tr = sum(loss_ops_tr) / num_processing_steps_tr

# Test/generalization loss.
loss_ops_ge = create_loss_ops(target_ph, output_ops_ge)
loss_op_ge = loss_ops_ge[-1]  # Loss from final processing step.

# Optimizer.
learning_rate = 1e-3
optimizer = tf.train.AdamOptimizer(learning_rate)
step_op = optimizer.minimize(loss_op_tr)

In [ ]:
# This cell resets the Tensorflow session, but keeps the same computational
# graph.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

last_iteration = 0
logged_iterations = []
losses_tr = []
corrects_tr = []
solveds_tr = []
losses_ge = []
corrects_ge = []
solveds_ge = []

In [ ]:
log_every_seconds = 20

print("# (iteration number), T (elapsed seconds), "
      "Ltr (training loss), Lge (test/generalization loss), "
      "Ctr (training fraction nodes/edges labeled correctly), "
      "Str (training fraction examples solved correctly), "
      "Cge (test/generalization fraction nodes/edges labeled correctly), "
      "Sge (test/generalization fraction examples solved correctly)")

start_time = time.time()
last_log_time = start_time
for iteration in range(last_iteration, num_training_iterations):
  last_iteration = iteration
  feed_dict, _ = create_feed_dict(rand, batch_size_tr, num_nodes_min_max_tr,
                                  theta, input_ph, target_ph)
  train_values = sess.run({
      "step": step_op,
      "target": target_ph,
      "loss": loss_op_tr,
      "outputs": output_ops_tr
  },
                          feed_dict=feed_dict)
  the_time = time.time()
  elapsed_since_last_log = the_time - last_log_time
  if elapsed_since_last_log > log_every_seconds:
    last_log_time = the_time
    feed_dict, raw_graphs = create_feed_dict(
        rand, batch_size_ge, num_nodes_min_max_ge, theta, input_ph, target_ph)
    test_values = sess.run({
        "target": target_ph,
        "loss": loss_op_ge,
        "outputs": output_ops_ge
    },
                           feed_dict=feed_dict)

## PyTorch Models

In [ ]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNLayer, self).__init__()
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3-5: Start propagating messages with "add" aggregation.
        return self.propagate('add', edge_index, x=x, num_nodes=x.size(0))

    def message(self, x_j, edge_index, num_nodes):
        # x_j has shape [E, out_channels]

        # Step 3: Normalize node features.
        row, col = edge_index
        deg = degree(row, num_nodes, dtype=x_j.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]

        # Step 5: Return new node embeddings.
        return aggr_out

In [ ]:
from torch_geometric.data import Data

# Sample Edge Label Definition for Rechits - Adjacency List?
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

In [ ]:
'''Define the 2-layer GCN'''

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, data.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)


In [ ]:
# CUDA available on cmg-gpu1080
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()